# Assignment II.
### *Stochastic Simulation*
#### Group 6
- Marcell Szegedi - 15722635
- Yuxin Dong - 15550397
- Koen Verlaan - 11848316

**Import Libraries**

In [11]:
import numpy as np
import simpy
import matplotlib.pyplot as plt
import random
from scipy import stats

**Output Libraries**

In [12]:
output_task_2 = r"Figures/task_2.png"
output_taks_3 = r"Figures/task_3.png"
output_task_4 = r"Figures/task_4.png"

# Models

##### M/M/n

In [7]:
class MMnQueueExperiment:
    def __init__(
            self,
            env: simpy.Environment,
            server: simpy.Resource,
            n_jobs: int,
            avg_arrival_time: float,
            avg_service_time: float):
        self.env = env
        self.server = server
        self.n_jobs = n_jobs
        self.avg_arrival_time = avg_arrival_time
        self.avg_service_time = avg_service_time
        self.waiting_times = []

    @classmethod
    def run_experiment(
            cls,
            n_servers: int,
            n_jobs: int,
            avg_arrival_time: float,
            avg_service_time: float
    ) -> np.ndarray:
        env = simpy.Environment()
        server = simpy.Resource(env, capacity=n_servers)
        experiment = cls(env, server, n_jobs, avg_arrival_time, avg_service_time)
        experiment.env.process(experiment.job_arrival())
        experiment.env.run()
        return np.array(experiment.waiting_times)

    def job_arrival(self):
        for _ in range(self.n_jobs):
            yield self.env.timeout(np.random.exponential(self.avg_arrival_time))
            self.env.process(self.job_life())

    def job_life(self):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request
            service_start_time = self.env.now
            yield self.env.timeout(np.random.exponential(self.avg_service_time))

        self.waiting_times.append(service_start_time - arrival_time)

##### M/D/n

In [8]:
class MDnQueueExperiment:
    def __init__(
            self,
            env: simpy.Environment,
            server: simpy.Resource,
            n_jobs: int,
            avg_arrival_time: float,
            det_service_time: float):
        self.env = env
        self.server = server
        self.n_jobs = n_jobs
        self.avg_arrival_time = avg_arrival_time
        self.det_service_time = det_service_time
        self.waiting_times = []

    @classmethod
    def run_experiment(
            cls,
            n_servers: int,
            n_jobs: int,
            avg_arrival_time: float,
            det_service_time: float
    ) -> np.ndarray:
        env = simpy.Environment()
        server = simpy.Resource(env, capacity=n_servers)
        experiment = cls(env, server, n_jobs, avg_arrival_time, det_service_time)
        experiment.env.process(experiment.job_arrival())
        experiment.env.run()
        return np.array(experiment.waiting_times)

    def job_arrival(self):
        for _ in range(self.n_jobs):
            yield self.env.timeout(np.random.exponential(self.avg_arrival_time))
            self.env.process(self.job_life())

    def job_life(self):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request
            service_start_time = self.env.now
            yield self.env.timeout(self.det_service_time)

        self.waiting_times.append(service_start_time - arrival_time)

##### M/H/n

In [9]:
class MHnQueueExperiment:
    def __init__(
            self,
            env: simpy.Environment,
            server: simpy.Resource,
            n_jobs: int,
            avg_arrival_time: float,
            p_exp_1: float,
            p_exp_2: float,
            avg_service_time_1: float,
            avg_service_time_2: float):
        self.env = env
        self.server = server
        self.n_jobs = n_jobs
        self.avg_arrival_time = avg_arrival_time
        self.p_exp_1 = p_exp_1
        self.avg_service_time_1 = avg_service_time_1
        self.p_exp_2 = p_exp_2
        self.avg_service_time_2 = avg_service_time_2
        self.waiting_times = []

    @classmethod
    def run_experiment(
            cls,
            n_servers: int,
            n_jobs: int,
            avg_arrival_time: float,
            p_exp_1: float,
            p_exp_2: float,
            avg_service_time_1: float,
            avg_service_time_2: float
    ) -> np.ndarray:
        env = simpy.Environment()
        server = simpy.Resource(env, capacity=n_servers)
        experiment = cls(env, server, n_jobs, avg_arrival_time, p_exp_1, p_exp_2, avg_service_time_1, avg_service_time_2)
        experiment.env.process(experiment.job_arrival())
        experiment.env.run()
        return np.array(experiment.waiting_times)

    def job_arrival(self):
        for _ in range(self.n_jobs):
            yield self.env.timeout(np.random.exponential(self.avg_arrival_time))
            self.env.process(self.job_life())

    def job_life(self):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request
            service_start_time = self.env.now
            yield self.env.timeout(self.hyper_exp_sampling(self.p_exp_1,
                                                           self.p_exp_2,
                                                           self.avg_service_time_1,
                                                           self.avg_service_time_2))

        self.waiting_times.append(service_start_time - arrival_time)

    @staticmethod
    def hyper_exp_sampling(p1: float, p2: float, avg_service_time_1: float, avg_service_time_2: float) -> float:
        if p1 + p2 != 1 or not (0 <= p1 <= 1) or not (0 <= p2 <= 1):
            raise ValueError("Probabilities must sum to 1.")
        if avg_service_time_1 <= 0 or avg_service_time_2 <= 0:
            raise ValueError("Parameters of the exponential distributions are not valid.")

        if random.random() < p1:
            return np.random.exponential(avg_service_time_1)
        else:
            return np.random.exponential(avg_service_time_2)

##### M/M/n with Priority to smaller jobs

In [6]:
class MMnPRIORITYQueueExperiment:
    def __init__(
            self,
            env: simpy.Environment,
            server: simpy.Resource,
            n_jobs: int,
            avg_arrival_time: float,
            avg_service_time: float):
        self.env = env
        self.server = server
        self.n_jobs = n_jobs
        self.avg_arrival_time = avg_arrival_time
        self.avg_service_time = avg_service_time
        self.waiting_times = []

    @classmethod
    def run_experiment(
            cls,
            n_servers: int,
            n_jobs: int,
            avg_arrival_time: float,
            avg_service_time: float
    ) -> np.ndarray:
        env = simpy.Environment()
        server = simpy.PriorityResource(env, capacity=n_servers)
        experiment = cls(env, server, n_jobs, avg_arrival_time, avg_service_time)
        experiment.env.process(experiment.job_arrival())
        experiment.env.run()
        return np.array(experiment.waiting_times)

    def job_arrival(self):
        for _ in range(self.n_jobs):
            yield self.env.timeout(np.random.exponential(self.avg_arrival_time))
            job_size = np.random.exponential(self.avg_service_time)
            self.env.process(self.job_life(job_size))

    def job_life(self, job_size: float):
        arrival_time = self.env.now
        with self.server.request(priority=job_size) as request:
            yield request
            service_start_time = self.env.now
            yield self.env.timeout(job_size)

        self.waiting_times.append(service_start_time - arrival_time)

# Hypothesis Testing

##### Functions

In [18]:
def one_tailed_z_test(sample_1: np.ndarray, sample_2: np.ndarray) -> float:
    """The function perform a one-tailed z-test for the difference of means of the two samples,
    where the null hypothesis is that the mean of the sample_1 is less than or equal to the mean of the sample_2."""
    mean_1 = np.mean(sample_1)
    mean_2 = np.mean(sample_2)
    var_1 = np.var(sample_1, ddof=1)
    var_2 = np.var(sample_2, ddof=1)
    
    se = np.sqrt(var_1 / len(sample_1) + var_2 / len(sample_2))

    z = (mean_1  - mean_2) / se
    p_value = 1 - stats.norm.cdf(z)

    return p_value

##### M/M/1 vs M/M/2

In [28]:
# Fixing seed for reproducibility
np.random.seed(100)
random.seed(100)

#####################

# Hypothesis testing for the MM1 vs MMn queues

# Test Settings
sample_size = 10000
cut_off_size = 1000
mean_arrival_time = 1
mean_service_time = 0.5

waiting_times_mm1 = MMnQueueExperiment.run_experiment(1, sample_size + cut_off_size, mean_arrival_time, mean_service_time)
waiting_times_mm2 = MMnQueueExperiment.run_experiment(2, sample_size + cut_off_size, mean_arrival_time, mean_service_time)

p_value = one_tailed_z_test(waiting_times_mm1[cut_off_size:], waiting_times_mm2[cut_off_size:])
print(f"The mean waiting time for MM1 is {np.mean(waiting_times_mm1[cut_off_size:]):.4f}")
print(f"The mean waiting time for MM2 is {np.mean(waiting_times_mm2[cut_off_size:]):.4f}")
print(f"p-value for the MM1 vs MM2 queue: {p_value:.4f}")

# Theoretical waiting time for MM1
rho = mean_service_time / mean_arrival_time
theoretical_mean_waiting_time = rho * mean_service_time / (1 - rho)
print(f"Theoretical mean waiting time for MM1: {theoretical_mean_waiting_time:.4f}")

The mean waiting time for MM1 is 0.4825
The mean waiting time for MM2 is 0.0345
p-value for the MM1 vs MM2 queue: 0.0000
Theoretical mean waiting time for MM1: 0.5000


##### M/M/1 vs M/M/4

In [29]:
# Fixing seed for reproducibility
np.random.seed(200)
random.seed(200)

#####################

# Hypothesis testing for the MM1 vs MMn queues

# Test Settings
sample_size = 10000
cut_off_size = 1000
mean_arrival_time = 1
mean_service_time = 0.5

waiting_times_mm1 = MMnQueueExperiment.run_experiment(1, sample_size + cut_off_size, mean_arrival_time, mean_service_time)
waiting_times_mm2 = MMnQueueExperiment.run_experiment(4, sample_size + cut_off_size, mean_arrival_time, mean_service_time)

p_value = one_tailed_z_test(waiting_times_mm1[cut_off_size:], waiting_times_mm2[cut_off_size:])
print(f"The mean waiting time for MM1 is {np.mean(waiting_times_mm1[cut_off_size:]):.4f}")
print(f"The mean waiting time for MM2 is {np.mean(waiting_times_mm2[cut_off_size:]):.4f}")
print(f"p-value for the MM1 vs MM2 queue: {p_value:.4f}")

The mean waiting time for MM1 is 0.5372
The mean waiting time for MM2 is 0.0002
p-value for the MM1 vs MM2 queue: 0.0000


##### M/M/2 vs M/M/4

In [30]:
# Fixing seed for reproducibility
np.random.seed(300)
random.seed(300)

#####################

# Hypothesis testing for the MM1 vs MMn queues

# Test Settings
sample_size = 10000
cut_off_size = 1000
mean_arrival_time = 1
mean_service_time = 0.5

waiting_times_mm1 = MMnQueueExperiment.run_experiment(2, sample_size + cut_off_size, mean_arrival_time, mean_service_time)
waiting_times_mm2 = MMnQueueExperiment.run_experiment(4, sample_size + cut_off_size, mean_arrival_time, mean_service_time)

p_value = one_tailed_z_test(waiting_times_mm1[cut_off_size:], waiting_times_mm2[cut_off_size:])
print(f"The mean waiting time for MM1 is {np.mean(waiting_times_mm1[cut_off_size:]):.4f}")
print(f"The mean waiting time for MM2 is {np.mean(waiting_times_mm2[cut_off_size:]):.4f}")
print(f"p-value for the MM1 vs MM2 queue: {p_value:.4f}")

The mean waiting time for MM1 is 0.0378
The mean waiting time for MM2 is 0.0001
p-value for the MM1 vs MM2 queue: 0.0000


##### M/M/1 priority vs M/M/1

##### M/M/1 priority vs M/M/4